<a href="https://colab.research.google.com/github/softmurata/colab_notebooks/blob/main/llm/llamav2langchain.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Installation

In [ ]:
!pip install llama-cpp-python
!pip install langchain
!pip install faiss-gpu
!pip install sentence_transformers

get Weights

In [ ]:
!wget https://huggingface.co/TheBloke/Llama-2-7B-Chat-GGML/resolve/main/llama-2-7b-chat.ggmlv3.q4_0.bin -P /content

In [ ]:
import logging
import sys

from langchain.chains import RetrievalQA
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.llms import LlamaCpp
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores.faiss import FAISS

# ログレベルの設定
logging.basicConfig(stream=sys.stdout, level=logging.DEBUG, force=True)

# ドキュメントの読み込み
with open("soccer.txt") as f:
    test_all = f.read()

# チャンクの分割
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=300,  # チャンクの最大文字数
    chunk_overlap=20,  # オーバーラップの最大文字数
)
texts = text_splitter.split_text(test_all)

# チャンクの確認
print(len(texts))
for text in texts:
    print(text[:10].replace("\n", "\\n"), ":", len(text))


# インデックスの作成
index = FAISS.from_texts(
    texts=texts,
    embedding=HuggingFaceEmbeddings(model_name="intfloat/multilingual-e5-large"),
)
index.save_local("storage")

# インデックスの読み込み
# index = FAISS.load_local(
#    "storage", HuggingFaceEmbeddings(model_name="intfloat/multilingual-e5-large")
# )

# LLMの準備
# llm = OpenAI(temperature=0, verbose=True)
llm = LlamaCpp(
    model_path="/content/llama-2-7b-chat.ggmlv3.q4_0.bin",
    input={
        "max_tokens": 32,
        "stop": ["System:", "User:", "Assistant:", "\n"],
    },
    verbose=True,
)

# 質問応答チェーンの作成
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=index.as_retriever(search_kwargs={"k": 4}),
    verbose=True,
)

Run QA Chain

In [ ]:
print("A1:", qa_chain.run("Tell us about the changes in the Japanese national team's play style."))

In [ ]:
print("A2:", qa_chain.run("I want you to tell me the results of the Japan national soccer team at the World Cup"))

翻訳

In [ ]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("facebook/nllb-200-1.3B")
model = AutoModelForSeq2SeqLM.from_pretrained("facebook/nllb-200-1.3B")

article = "UN Chief says there is no military solution in Syria"
inputs = tokenizer(article, return_tensors="pt")

translated_tokens = model.generate(
    **inputs, forced_bos_token_id=tokenizer.lang_code_to_id["jpn_Jpan"], max_length=30
)
ret = tokenizer.batch_decode(translated_tokens, skip_special_tokens=True)[0]
print(ret)

In [ ]:
# eng_Latn 英語に変換するには
prompt = "おはようございます。今日はいい天気ですね。"
inputs = tokenizer(article, return_tensors="pt")

translated_tokens = model.generate(
    **inputs, forced_bos_token_id=tokenizer.lang_code_to_id["eng_Latn"], max_length=30
)
ret = tokenizer.batch_decode(translated_tokens, skip_special_tokens=True)[0]
print(ret)